In [1]:
from keras import models
from keras.layers import *
import os
import cv2
from matplotlib import pyplot as plt

Using TensorFlow backend.
/home/rahil/anaconda3/envs/tenserflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rahil/anaconda3/envs/tenserflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rahil/anaconda3/envs/tenserflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rahil

In [2]:
#reading and preprocessing input
os.chdir('/home/rahil/image-Segmentation/Datasets')
#reading x and y data
x=np.load('Skin_TestX.npy')
y=np.load('Skin_TestY.npy')
#reading the test_data
x_test=np.load('Skin_ValidX.npy')
y_test=np.load('Skin_ValidY.npy')

In [3]:
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)

(600, 192, 256, 3)
(600, 192, 256)
(150, 192, 256, 3)
(150, 192, 256)


In [4]:
#preprocessing of data
#converting the data into first gray scale and then reducing dimensions
x_gray=[]
for i in range(0,len(x)):
    x_gray.append(cv2.cvtColor(x[i],cv2.COLOR_BGR2GRAY))
x=np.array(x_gray)
x_test_gray=[]
for i in range(0,len(x_test)):
    x_test_gray.append(cv2.cvtColor(x_test[i],cv2.COLOR_BGR2GRAY))
x_test=np.array(x_test_gray)

del x_gray
del x_test_gray

#resizing of the array into size of 128, 128 using cv2
x_resize=[]
for i in range(0,len(x)):
    x_resize.append(cv2.resize(x[i],(128,128)))
print(len(x_resize))
x_tresize=[]
for i in range(0,len(x_test)):
    x_tresize.append(cv2.resize(x_test[i],(128,128)))
print(len(x_tresize))

x=np.array(x_resize)
print(x.shape)
x_test=np.array(x_tresize)
print(x_test.shape)

#now converting each element into 3 dimension
x=x.reshape(-1,128,128,1)
print(x.shape)
x_test=x_test.reshape(-1,128,128,1)
print(x_test.shape)

600
150
(600, 128, 128)
(150, 128, 128)
(600, 128, 128, 1)
(150, 128, 128, 1)


In [5]:
#now performing the same operations for the y data
y_resize=[]
y_tresize=[]
for i in range(0,len(y)):
    y_resize.append(cv2.resize(y[i],(128,128)))
len(y_resize)
y=np.array(y_resize)
print(y.shape)
y=y.reshape(-1,128,128,1)
print(y.shape)

for i in range(0,len(y_test)):
    y_tresize.append(cv2.resize(y_test[i],(128,128)))
len(y_tresize)
y_test=np.array(y_tresize)
print(y_test.shape)
y_test=y_test.reshape(-1,128,128,1)
print(y_test.shape)

del y_resize
del y_tresize
del x_resize
del x_tresize

(600, 128, 128)
(600, 128, 128, 1)
(150, 128, 128)
(150, 128, 128, 1)


In [6]:
#finally after performing the all transformation
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)

(600, 128, 128, 1)
(600, 128, 128, 1)
(150, 128, 128, 1)
(150, 128, 128, 1)


In [7]:
#performing the normalization of taking each value between 0 and 1
x=x.astype('float32')/255
y=y.astype('float32')/255
x_test=x_test.astype('float32')/255
y_test=y_test.astype('float32')/255

In [8]:
#normalizing the training data
x_mean=np.mean(x)
x_std=np.std(x)
x=x-x_mean
x=x/x_std
#normalizing the testing data 
#x_test=x_test-x_mean
#x_test=x_test/x_std

In [13]:
#creating a FCNN Model with constant kernel size results
model1 =models.Sequential()
model1.add(Conv2D(16, (3,3), activation='relu',   padding='same',input_shape=(128,128,1)))
model1.add(Conv2D(32, (3,3), activation='relu',   padding='same'))
model1.add(Conv2D(64, (3,3), activation='relu',   padding='same'))
model1.add(Conv2D(128,(3,3),activation='relu', padding='same'))
model1.add(Dropout(0.5))
model1.add(Conv2D(1,  (1,1), activation='sigmoid',   padding='same'))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 16)      160       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 128, 128, 1)       129       
Total params: 97,281
Trainable params: 97,281
Non-trainable params: 0
_________________________________________________________________


In [14]:
model1.compile(optimizer='adam',loss='mse',metrics=['accuracy'])

In [15]:
#creating callbacks
#creating the callbacks and checkpoints to save best model
from keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor="val_loss",mode='min',patience=10 ,verbose=1)
cp=ModelCheckpoint('best_model_FCNN1.h5',monitor="val_loss",mode='min',verbose=1,save_best_only='true')

In [ ]:
#fitting the model
history=model1.fit(x,y,batch_size=32,epochs=50,validation_split=0.2,callbacks=[es,cp],verbose=1,shuffle='true',class_weight='auto')

Train on 480 samples, validate on 120 samples
Epoch 1/50
 32/480 [=>............................] - ETA: 15:47 - loss: 0.2474 - acc: 0.5127